In [42]:
import numpy as np
import scipy.io as spio
import scipy.stats as stats
import math
import time

class GaussianProbabilisticModel:
    
    def __init__(self):
        self.num_classes = 0
        self.classes = []
        self.mle_mu_map = {}
        self.mle_sigma_map = {}
        self.class_prob_map = {}
        self.dim = 0
        self.topN_features = 200
        self.l = 0.1

    def __log_gaussian_pdf(self,x,mu,sigma,sigma_new_I_given = None,logdet_given = None,compute_inv_and_det = True):
        # calculate log of f(x)~multivariate Normal density
        # sigma is adjusted by 0.1*I to avoid inverting a singular matrix
        dim_of_cut_data = len(mu)
        sigma_new = np.matrix(sigma+self.l*np.eye(dim_of_cut_data))
        
        # can feed the inverse and det in directly so that we don't have to compute it every time
        if (compute_inv_and_det):
            sigma_new_I = sigma_new.I
            (sign, logdet) = np.linalg.slogdet(sigma_new)
        else:
            sigma_new_I = sigma_new_I_given
            logdet = logdet_given
        
        C = -0.5*logdet + (-dim_of_cut_data*0.5)*np.log(2*math.pi)
        x_minus_mu_mat = np.matrix(x-mu).T
        result = float(-0.5*x_minus_mu_mat.T*sigma_new_I*x_minus_mu_mat)

        return result
                
       
    def train_model(self,training_data,labels):
        # Steps:
        # 1: Find the N=200 most significant features, and slice the data based on those features
        # 2: Partition data according to labels
        # 3: Compute MLE of mu and sigma for each label
        
        # Step 1.1 - find the MLE of mu and sigma for each feature over the entire dataset
        for i in range(len(training_data)):
            if (i == 0):
                feature_mle_mu = training_data[i]
                self.dim = (training_data[i].shape)[0]
            else:
                feature_mle_mu += training_data[i]
            
        feature_mle_mu = feature_mle_mu * 1.0/len(training_data)
        
        for i in range(len(training_data)):
            x_minus_mu_feature = training_data[i]-feature_mle_mu
            if (i == 0):
                feature_mle_sigma = x_minus_mu_feature**2
            else:
                feature_mle_sigma += x_minus_mu_feature**2
                
        feature_mle_sigma = feature_mle_sigma * 1.0/len(training_data)
        
        # Step 1.2 - find the N=200 most significant features (features exhibiting the largest variance)
        keys = np.linspace(0,self.dim-1,self.dim)
        feature_var = list(zip(keys,feature_mle_sigma))
        
        feature_var_sorted = sorted(feature_var, key=lambda f: f[1], reverse = True)
        self.topN = [int(e[0]) for e in feature_var_sorted[:self.topN_features]]  
        
        # Step 1.3 - slice the training data on the most significant features
        training_data = training_data[:,self.topN]
        self.train_size = len(training_data)
        nums={}
        
        # Step 2.1 - calculate MLE of mu for each label
        for i in range(len(training_data)):
            if not (labels[i] in self.mle_mu_map):
                self.mle_mu_map[labels[i]] = training_data[i]
                self.classes += [labels[i]]
                nums[labels[i]] = 1
                self.num_classes += 1
                
            else:
                self.mle_mu_map[labels[i]] += training_data[i]
                nums[labels[i]] += 1
                
        for i in self.classes:
            self.mle_mu_map[i]=(1.0/nums[i])*self.mle_mu_map[i]
        
        # Step 2.2 - calculate MLE of sigma for each label
        for i in range(len(training_data)):
            # this is a column vector
            x_minus_mu = np.matrix(training_data[i]-self.mle_mu_map[labels[i]]).T
            
            if not (labels[i] in self.mle_sigma_map):
                self.mle_sigma_map[labels[i]]=(x_minus_mu)*(x_minus_mu.T)
            else:
                self.mle_sigma_map[labels[i]]+=(x_minus_mu)*(x_minus_mu.T)
               
        for i in self.classes:
            self.mle_sigma_map[i]=(1.0/nums[i])*self.mle_sigma_map[i]
            self.class_prob_map[i]=(nums[i]*1.0)/len(training_data)
         
        # precalculate matrix inverses and determinants to use in prediction function
        # the "new" refers to computing the inverse of sigma + l*I 
        self.sigma_new_inverses = {}
        self.logdets = {}
        for i in self.classes:
            self.sigma_new_inverses[i]=np.matrix(self.mle_sigma_map[i] + self.l*np.eye(self.topN_features)).I
            self.logdets[i]=np.linalg.slogdet(np.matrix(self.mle_sigma_map[i] + self.l*np.eye(self.topN_features)))[1]
            
    def predict(self,x):
        # prediction is equal to the maximum likelihood class
        prediction=-1
        curr_max=-1e300;
        
        for i in self.classes:
            p = self.__log_gaussian_pdf(x,self.mle_mu_map[i],self.mle_sigma_map[i],
                                       sigma_new_I_given = self.sigma_new_inverses[i],
                                       logdet_given = self.logdets[i],
                                       compute_inv_and_det = False) + np.log(self.class_prob_map[i])
            if(p>curr_max):
                curr_max=p
                prediction=i
        return prediction
                        
    def test_model(self,test_data,labels):
        misses=0        
        # slice the data based on the most significant features
        test_data = test_data[:,self.topN]
        self.test_size = len(test_data)
        results = {'Test Error': [], 'Training Size': [], 'Testing Size': [], 'Predicted': [], 'Actual': []}
        results['Training Size'] = self.train_size
        results['Testing Size'] = self.test_size

        for i in range(len(test_data)):
            pred = self.predict(test_data[i])
            actual = labels[i]
            
            results['Predicted'] += [pred]
            results['Actual'] += [actual]
            
            if(pred != actual):
                misses=misses+1
                
        results['Test Error'] = misses*1.0/len(test_data)
        return results


mat = spio.loadmat('hw1data.mat', squeeze_me=True)
image_matrix=mat['X']
label_array=mat['Y']   

tstart = time.time()
    
gaussian_model= GaussianProbabilisticModel()
gaussian_model.train_model(image_matrix[:3000,:],label_array[:3000])
tmid = time.time()
print('Training complete')
print(gaussian_model.test_model(image_matrix[3001:3301,:],label_array[3001:3301]))

tend = time.time()
print("Training time: " + str(tmid-tstart))
print("Testing time: " + str(tend-tmid))
print("Total time taken: " + str(tend-tstart))



Training complete
{'Test Error': 0.23333333333333334, 'Training Size': 3000, 'Testing Size': 300, 'Predicted': [5, 8, 9, 0, 0, 9, 0, 8, 8, 3, 8, 0, 6, 9, 2, 8, 7, 9, 4, 3, 1, 5, 8, 8, 1, 9, 9, 5, 3, 8, 8, 2, 5, 3, 4, 5, 4, 0, 8, 0, 3, 6, 2, 8, 8, 3, 8, 8, 9, 3, 8, 9, 9, 8, 7, 1, 2, 9, 3, 0, 2, 0, 8, 9, 4, 7, 3, 1, 4, 3, 3, 1, 3, 1, 2, 8, 6, 8, 3, 8, 5, 1, 3, 3, 6, 8, 3, 2, 3, 3, 7, 3, 3, 3, 8, 3, 8, 2, 8, 9, 1, 7, 9, 3, 0, 1, 3, 6, 5, 6, 3, 3, 3, 7, 1, 7, 9, 8, 9, 1, 8, 4, 8, 8, 4, 1, 6, 3, 8, 7, 9, 5, 3, 4, 2, 0, 6, 0, 4, 8, 3, 8, 2, 7, 8, 9, 8, 6, 5, 8, 8, 0, 0, 9, 8, 0, 1, 1, 6, 9, 5, 3, 2, 3, 8, 8, 2, 8, 7, 9, 7, 3, 0, 4, 3, 0, 4, 7, 8, 9, 3, 1, 8, 4, 3, 4, 3, 5, 9, 3, 0, 2, 3, 9, 8, 2, 4, 3, 2, 3, 3, 0, 8, 3, 7, 0, 3, 4, 3, 8, 8, 7, 0, 8, 6, 8, 2, 7, 8, 3, 0, 9, 3, 4, 7, 9, 5, 8, 3, 3, 9, 3, 3, 8, 9, 2, 2, 1, 5, 3, 7, 5, 4, 8, 8, 3, 7, 8, 1, 1, 6, 3, 9, 7, 9, 1, 9, 4, 8, 9, 2, 9, 3, 6, 2, 8, 1, 0, 7, 2, 4, 9, 3, 7, 3, 0, 5, 5, 6, 4, 1, 2, 2, 8, 4, 0, 3, 7, 7, 1, 8, 3, 8, 1, 9, 8, 